# Cohort Analysis Project – IronHack Payments

## 📌 Introduction

IronHack Payments, founded in 2020, offers innovative cash advance solutions with transparent pricing and no hidden fees. This project aims to uncover user behavior patterns using cohort analysis, helping IronHack Payments further optimize its services.

---

## 🧭 Project Overview

The goal is to analyze user cohorts defined by the month in which users made their **first cash advance request**. By tracking key metrics over time per cohort, we aim to provide actionable insights into usage behavior and financial performance.

---

## 🎯 Metrics to Analyze

We will compute and analyze the following metrics for each cohort:

1. **Frequency of Service Usage**
    Understand how often users from each cohort utilize IronHack Payments' cash advance services over time.

2. **Incident Rate**
    Determine the incident rate, specifically focusing on payment incidents, for each cohort. Identify if there are variations in incident rates among different cohorts.

3. **Revenue Generated by the Cohort**
    Calculate the total revenue generated by each cohort over months to assess the financial impact of user behavior.

4. **New Relevant Metric (Optional)**
   Propose and calculate a new relevant metric that provides additional insights into user behavior or the performance of IronHack Payments' services.

---

## 🛠️ Data Analysis Tools

- **Language:** Python (primary)
- **Libraries:**
  - `pandas` for data cleaning and manipulation
  - `matplotlib` / `seaborn` for visualization
---

## 🔎 Exploratory Data Analysis (EDA)

Before conducting the cohort analysis:

- Inspect dataset structure and column types
- View summary statistics using `.describe()`
- Explore distributions and detect outliers
- Examine timestamp formats and identify feature gaps

Example EDA:
- Count of cash requests per month
- Distribution of advance amounts
- Frequency of payment incidents

---

## 🧹 Data Quality Analysis

Check for and address:

- Missing values (`.isnull().sum()`)
- Duplicate records
- Inconsistent or malformed dates
- Incorrect data types (e.g., strings instead of numeric)

Document all issues found and how they were handled (e.g., dropped rows, converted types, filled values).

---

## ✅ Deliverables

- **Python Code**:
  - Data loading
  - Data cleaning and preprocessing
  - Cohort assignment and metric calculations
  - Visualization of trends and patterns

- **Visualizations**:
  - Usage frequency over time
  - Incident rate per cohort
  - Revenue heatmap
  - Custom metric visualization

- **Optional**:
  - Written summary or interpretation of findings
  - Recommendations based on results

---

## Table of Contents

1. [🛠️ Data Analysis Tools](#️-data-analysis-tools)
2. [📥 Importing Data](#-importing-data)
3. [🔎 Exploratory Data Analysis (EDA)](#-exploratory-data-analysis-eda)
4. [🧹 Data Cleaning / Quality Analysis](#-data-cleaning--quality-analysis)  
5. [🔎 Exploratory Data Analysis (EDA) – Post-Cleanup](#-exploratory-data-analysis-eda--post-cleanup)
6. [🎯 Metrics to Analyze](#-metrics-to-analyze)
    - [Frequency of Service Usage](#frequency-of-service-usage)
    - [Incident Rate](#incident-rate)
    - [Revenue Generated by the Cohort](#revenue-generated-by-the-cohort)




## 🛠️ Data Analysis Tools

In [556]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

### Configure libraries

In [557]:
# Set the Seaborn context to "poster" for larger text and figures
sns.set_context("poster")

# Set the default figure size for Seaborn plots
sns.set_theme(rc={"figure.figsize": (12., 6.)})

# Set the Seaborn style to "whitegrid" for a white background with gridlines
sns.set_style("whitegrid")

## 📥 Importing Data

In [558]:
#First we removed the spaces from the csv files so we can easily import them here

# lists of columns containing dates -> for parsing
datetime_columns_cash_request = [
    "created_at",
    "updated_at",
    "moderated_at",
    "cash_request_received_date",
    "reimbursement_date",
    "money_back_date",
    "send_at",
    "reco_last_update",
    "reco_creation"
]
   
datetime_columns_fees = [
    "created_at",
    "updated_at",
    "paid_at",
    "from_date",
    "to_date"
]

# We modified our import process to directly cast proper datatypes for dates.
# Float/integer will still be handled in data cleaning, 
# since some of the offending columns have NaN values causing issues (hence presumably the wrong automatic casting)
fees = pd.read_csv("../project_dataset/extract-fees-dataanalyst.csv",
                            parse_dates = datetime_columns_fees)
cashRequest = pd.read_csv("../project_dataset/extract-cashrequest-dataanalyst.csv", 
                            parse_dates = datetime_columns_cash_request)



## 🔎 Exploratory Data Analysis (EDA)

In [559]:
# This is how we get a small insight in the data
display(fees.head())
# Overview of data in fees
fees.info()

**Observations**:

- `cash_request_id` is automatically cast as float64. `int` might be more plausible, change in cleaning
- date-related columns (`created_at`,`updated__at`,`paid_at`,`from_date`,`to_date`) will need special treatment 
- after date casting at import still trouble for `paid_at`,`from_date`,`to_date`

In [560]:
display(cashRequest.head())
# Overview of data in cashRequest
cashRequest.info()

**Observations**:

- `delete_account_id` and `user_id` needn't be floats (cast to int later)
- date-related columns (`created_at`,`updated__at`,`moderated_at`...) will need special treatment
- fewer unique `user_id` values than cashRequest `id`s: indicating multiple transactions for some users or actual missing values?
- after date casting at import the following fields are still `object` rather than `datetime`:
  `moderated_at`,`reimbursement_date`,`money_back_date`,`send_at`


## Function for evaluations

In [561]:
# One function to show all details about the dataframe
def evaluateDataFrame(df):
    # Lets check how many values we actually have
    print("Total amount of records")
    print(len(df))
    print()
    # This shows us the amount of empty rows for each column 
    print("Empty rows")
    print(df.isna().sum())
    print()
    # check the number of unique values for each column 
    print("Unique rows")
    print(df.nunique())
    print()
    print("DataFrame info")         
    fees.info()
    print()

### Use evaluateDataFrame to evaluate our data

In [562]:

# commented for now, piecewise presentation might be more readable in notebook 

evaluateDataFrame(cashRequest)
evaluateDataFrame(fees)


### Use inspect_data_types to get to know the differnce datatypes
Only two types of fees are levied: 5 and 10 Euros(?) - maybe convert to int as well?

In [563]:
def inspect_data_types(df, name="DataFrame"):
    print(f"=== {name} ===")
    numerical = df.select_dtypes(include='number').columns.tolist()
    categorical = df.select_dtypes(include='object').columns.tolist()
    datetime = df.select_dtypes(include=['datetime','datetime64','datetime64[ns, UTC]']).columns.tolist()
      
    print(f"Numerical columns ({len(numerical)}): {numerical}")
    print(f"Categorical columns ({len(categorical)}): {categorical}")
    print(f"Date columns ({len(datetime)}): {datetime}")
    print()
    
    return numerical, categorical, datetime       


In [564]:

cashr_numcols, cashr_strcols, cashr_dtcols = inspect_data_types(cashRequest, name="cashRequest")


In [565]:
fees_numcols, fees_strcols, fees_dtcols = inspect_data_types(fees, name="fees")


Several of the date fields aren't typed correctly, fix in data cleaning and rerun function

**Observations**

- 2103 empty values in `cashRequest.user_id` corresponding to the difference to `id` noted above
  - also: very close to value of `deleted_account` id (2104), so possible relation to that
- fees are associated to cashRequests via `cash_request_id`

We used these insights to adapt our data import in order to directly cast the correct datatypes for columns that were not correctly identified automatically.  

## 🧹 Data Cleaning / Quality Analysis

### Instructions after EDA
1. Parse all values to the right data types
2. remove loose items (like fees without cashRequest)
3. Fill the user_id columns with deleted_account_id were empty because of account deletion

In [566]:
# clean the start and ends of all column names so we get no suprises in the data retrieval later
fees.columns = fees.columns.str.strip()
cashRequest.columns = cashRequest.columns.str.strip()

In [567]:
cashRequest['user_id'] = cashRequest['user_id'].fillna(cashRequest['deleted_account_id'])

print(cashRequest['user_id'].isnull().sum())

The next block is going to fix datatypes for both dataframes, i.e. fixing the missing dates and casting some columns as integers.

In [568]:

# This is antoher option of parsing datatypes
# errors="coerce" -> means that erroes will force conversion and replace any invalid or unconvertible values with NaT



datetime_columns_cash_request = [
    "created_at",
    "updated_at",
    "moderated_at",
    "cash_request_received_date",
    "reimbursement_date",
    "money_back_date",
    "send_at",
    "reco_last_update",
    "reco_creation"
]

for col in datetime_columns_cash_request:
    cashRequest[col] = pd.to_datetime(cashRequest[col], errors="coerce")
    
# the cash_request_received_date doesnt have an timezone so we normalize it to the standard utc
cashRequest["cash_request_received_date"] = cashRequest["cash_request_received_date"].dt.tz_localize("UTC")
datetime_columns_fees = [
    "created_at",
    "updated_at",
    "paid_at",
    "from_date",
    "to_date"
]

for col in datetime_columns_fees:
    fees[col] = pd.to_datetime(fees[col], errors="coerce")
    


float_to_int_fees = [
    "cash_request_id",
    "id"
]

# This currently doesn't work with astype(int), while astype("Int64")
for col in float_to_int_fees:
    fees[col] = pd.to_numeric(fees[col], errors="coerce").astype("Int64")
     
float_to_int_cash_request = [
    "user_id",
    "deleted_account_id",
    "id"
]

for col in float_to_int_cash_request:
    cashRequest[col] = pd.to_numeric(cashRequest[col], errors="coerce").astype("Int64")
    
fees.info()


In [570]:

cashr_numcols, cashr_strcols, cashr_dtcols = inspect_data_types(cashRequest, name="cashRequest")

In [572]:
fees_numcols, fees_strcols, fees_dtcols = inspect_data_types(fees, name="fees")


### Cleaning floats that should be ints

In [573]:
fees[fees['cash_request_id'].isna()]

NA-values in `fees.cash_request_id` are for cancelled transactions - let's drop them!(?)

In [574]:
# creating copies before dropping values (optional)
fees_cp = fees.copy()
cashRequest_cp = cashRequest.copy()

In [575]:
# Drop all rows that have are not connected to a cash request anymore
fees_cp.dropna(subset=['cash_request_id'],inplace=True)

### Checking NaT dates

In [ ]:
# check date fields with missing data to assess significance
for col in cashr_dtcols:
    if cashRequest_cp[col].isna().sum() > 0:
        print(col, ': ', cashRequest_cp[col].isna().sum())       

### Creating column for cohorts

In [578]:
# create a new column with the created_at converted to the month annotation like 2024-03-18, 2023-11-02 etc -> these can be used to be labels for the plot

#add cohort month-assigns each row to a monthly cohort based on when the request was created (above) 
    #basically this just adds another column in the excel sheet called "cohort"
cashRequest_cp['created_month'] = cashRequest_cp['created_at'].dt.to_period('M')

cashRequest_cp['cohort'] = (
    cashRequest_cp
    .groupby('user_id')['created_month']
    .transform('min')  # the user's first month
)


## 🔎 Exploratory Data Analysis (EDA) – Post-Cleanup

### Numerical data

#### Cash Requests

In [580]:
cashRequest_cp.describe()

`amount` is the only numerical variable where the statistics are meaningfully interpretable, since the others are just IDs.
Cash request amounts ranged between 1€ and 200€ with the median at 100€ and a mean of just under 83€.

The histogram below illustrates this insight, with the majority of datapoints at 100€ and most of the remainder beneath that value.

The further split-up by status also shows that almost all of the small amount requests (around 25€) have been successfully reimbursed. For amounts of 50 and 100, there is a noticable proportion of cancelled requests (probably just over half for 50€ and about 1/3 for the 100€ requests).

In [581]:
fees_cp.describe()

Fees ranged between 5 and 10€ with most of them at 5€ (given that the mean is almost at 5€).

In [582]:
sns.histplot(fees_cp,x='status',palette='colorblind', hue='status');


Most of the fees were successfully charged. A smaller number of fee bookings were cancelled (following the documentation mostly due to failed postpone fees, so there might be an expected correlation with `type` postpone). An even smaller number of fees had the `status` rejected, i.e. their charge had failed.

In [583]:
pd.crosstab(fees_cp.type,fees_cp.status)

Indeed, the overwhelming majority of rows with `status` cancelled are of `type` postpone. This is also reflected in the barplot below.

In [584]:
# total fees by type of fee
sns.barplot(fees_cp,x='type',y='total_amount',hue='status',palette='colorblind',estimator=sum)

The major amount of fees comes from `instant_payment`s, a smaller amount from `postponement`s (of which more than half are actually cancelled) and the smallest amount comes from `incident`s

## Categorical data

In [585]:
cr_status_counts = cashRequest_cp.status.value_counts()
cr_status_counts

In [591]:
# Group fees per Cash request and count the amount of fees for the cash request
fee_counts = fees_cp.groupby("cash_request_id").size().reset_index(name="fee_count")

# Visualizing
plt.figure(figsize=(10, 6))
sns.histplot(fee_counts["fee_count"], bins=range(1, fee_counts["fee_count"].max() + 2), kde=False)
plt.title("Aantal fees per cash_request_id")
plt.xlabel("Aantal fees")
plt.ylabel("Aantal cash_request_id's")
plt.xticks(range(1, fee_counts["fee_count"].max() + 1))
plt.grid(axis="y")
plt.show()

In [592]:
# histogram of amounts for cashrequests (not using the merged dataset here to avoid issues )
sns.histplot(cashRequest_cp,x="amount")

Most common cash advance requests were for 100€, followed by 50€ and some 25€ requests.

In [596]:
# plotting histogram of amount
sns.histplot(cashRequest_cp,x="amount",y="status")


## 🎯 Metrics to Analyze


### Frequency of Service Usage

In [598]:

# Group data
cohort_counts = cashRequest_cp.groupby('cohort')['user_id'].count()

# display(cohort_counts)
created_counts = cashRequest_cp.groupby('created_month')['user_id'].count()
# Convert Period to string if needed
cohort_counts.index = cohort_counts.index.astype(str)
created_counts.index = created_counts.index.astype(str)

# Align indexes
combined = pd.DataFrame({
    'Cohort': cohort_counts,
    'Created': created_counts
}).fillna(0)  # in case some months are missing in one of the series

# Plot as grouped bar chart
ax = combined.plot(
    kind='bar',
    figsize=(14, 6),
    width=0.8,  # controls total bar width
    color=['steelblue', 'orange'],
    title='User Counts: Cohort vs Created Month'
)

plt.xlabel('Month')
plt.ylabel('Number of Users')
plt.xticks(rotation=45)
plt.legend(title='Type')
plt.tight_layout()
plt.grid(axis='y')
plt.show()

In [ ]:
# get counts for users per cohort (courtesy of Jan above)
cohort_counts = cashRequest_cp.groupby('cohort')['user_id'].nunique().reset_index()
cohort_counts.rename(columns={'user_id':'user_count'},inplace=True)
cohort_counts['cohort'] = cohort_counts['cohort'].astype(str)
cohort_counts.dtypes

In [ ]:
# create groupby for cohort and month created
cashReq_grouped = cashRequest_cp.groupby(['cohort', 'created_month'])['id'].count().reset_index()
cashReq_grouped['cohort'] = cashReq_grouped['cohort'].astype(str)

# renaming
cashReq_grouped.rename(columns={'id': 'request_count'}, inplace=True)

# merging with cohort_counts to get user numbers per cohort
cashReq_grouped= cashReq_grouped.merge(cohort_counts,on='cohort',how='left')
# calculate interactions per user
cashReq_grouped['interactions_per_user'] = round(cashReq_grouped['request_count'] / cashReq_grouped['user_count'],3)

# retyping and reformatting
cashReq_grouped['created_month_dt'] = cashReq_grouped['created_month'].dt.to_timestamp()
cashReq_grouped['created_month_str'] = cashReq_grouped['created_month_dt'].dt.strftime('%Y-%m')

# sorting
cashReq_grouped.sort_values('created_month_dt', inplace=True)

cashReq_grouped.dtypes

Lineplot and barplot are suboptimal, especially due to the large numbers-

In [599]:

## Create the lineplot
sns.lineplot(
    data=cashReq_grouped,
    x='created_month_str',
    y='request_count',
    hue='cohort'
)

# layout
plt.title('Cash Requests Over Time by Cohort')
plt.xlabel('Created Month')
plt.ylabel('Number of Requests')
plt.xticks(rotation=45)
plt.tight_layout()

plt.show()


In [ ]:

## Create the lineplot
sns.barplot(
    data=cashReq_grouped,
    x='created_month_str',
    y='request_count',
    hue='cohort'
)

# layout
plt.title('Cash Requests Over Time by Cohort')
plt.xlabel('Created Month')
plt.ylabel('Number of Requests')
plt.xticks(rotation=45)
plt.tight_layout()

plt.show()

A heatmap representation allows representation of interaction counts per month for each cohort:

In [601]:
pivot = cashReq_grouped.pivot(index='cohort', columns='created_month_str', values='request_count')
pivot = pivot.astype(float).fillna(0)  # Replace NaNs with 0

sns.heatmap(pivot, cmap='Greens', annot=True, fmt='g')
plt.title('Cash Request Utilisation by Cohort over Time')
plt.xlabel('Month')
plt.ylabel('Cohort')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

Alternatively, we can relativise the interaction counts to the cohort size to account for different total numbers. The diagonal values are at least 1 because the number of cohort members gets instated by their first interaction with the system. The very small first column automatically gets very "strong" values even though a cohort of 1 member may not be that informative.

In [602]:
pivot = cashReq_grouped.pivot(index='cohort', columns='created_month_str', values='interactions_per_user')
pivot = pivot.astype(float).fillna(0)  # Replace NaNs with 0

sns.heatmap(pivot, cmap='Greens', annot=True, fmt='g')
plt.title('Cash Request Utilisation per User by Cohort over Time')
plt.xlabel('Month')
plt.ylabel('Cohort')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [603]:
# alternative attempt, can maybe be ignored
sns.histplot(cashReq_grouped,x='created_month_str',y='request_count',hue='cohort')

### Incident Rate


In [ ]:
# get a collection of all fees and left join the cashRequest data on it
cashRequestFees = fees.merge(cashRequest_cp, how='left', left_on='cash_request_id', right_on='id')

# Count number of items per (cohort, type)
incident_count = cashRequestFees.groupby(['cohort', 'type']).size().unstack(fill_value=0)
print("incident count:")
display(incident_count)
print()

# Get total rows per cohort
cohort_totals = incident_count.sum(axis=1)
print("Cohort totals:")
display(cohort_totals)
print()

# Calculate incident rate per status (per cohort)
incident_rate = incident_count.divide(cohort_totals, axis=0)
print("incident_rate:")
print(incident_rate.head())
print()

# Clean up column names: remove underscores and make it a title
incident_rate.columns = [col.replace('_', ' ').title() for col in incident_rate.columns]
print("columns:")
print(incident_rate.columns)
print()

# Plot as stacked bar chart
incident_rate.plot(
    kind='bar',
    stacked=True,
    figsize=(14, 6),
    colormap='tab20',
    title='Incident Rate per Status per Cohort Month'
)

plt.xlabel('Cohort Month')
plt.ylabel('Incident amount')
plt.legend(title='Status', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=45)
plt.tight_layout()
plt.grid(axis='y')
plt.show()

### Revenue Generated by the Cohort

In [ ]:
##3. **Revenue Generated by the Cohort:** Calculate the total revenue generated by each cohort over months to assess the financial impact of user behavior.
# Merge revenue info from fees: 
    #1. make smaller version of the fees DF that has: which cash_request_id was charged, how much was charged (total_amount)
    #2.renames cash_request_id to match the id column in cashRequest
    #3.Merges them w my copy of cashRequest  (if the ids match) to add a new column (total_amount from fees) to the cashrequest copy. Merged left so we keep all the cash requests even if they are NaN for fees.  
fees_cp = fees[["cash_request_id", "total_amount"]].copy()
fees_cp = fees_cp.rename(columns={"cash_request_id": "id"})
cashRequest_cp = cashRequest_cp.merge(fees_cp, on="id", how="left")

# Add request month column to cashrequstcp: tracks when each cash request was made (monthly) and allows for 2D table;
    #request_month is specific to each cash request so its specific to each transaction! It can repeat if users make multipe requests each month and it can very for the same user in different months. 
        #we need this to analyze the total revenue generated by each cohort OVER MONTHS. WIthout it we only know the revenue from each cohort!
    #cohort is specific to each user and doesnt change across rows for the same user. We need this to group users into cohorts.
    #2D table: #rows (when joined-cohort, columns-when request was made, values-revenue in that month)
    #allows you to analyze how revenue changes over time for each group of created_at (the cohort)
cashRequest_cp["request_month"] = cashRequest_cp["created_at"].dt.to_period("M")
##3. **Revenue Generated by the Cohort:** Calculate the total revenue generated by each cohort over months to assess the financial impact of user behavior.

#tells you how many requests (txns) were made each month
cashRequest_cp["request_month"].value_counts().sort_index()


In [ ]:
##3. **Revenue Generated by the Cohort:** Calculate the total revenue generated by each cohort over months to assess the financial impact of user behavior.

#VISUALIZATION
# Group and calculate total revenue (total_amount) per cohort per request month
    #groups data by: cohort (when joined)
    #request_month (when revenue happened)

revenue_by_cohort = (
    cashRequest_cp
    .groupby(["cohort", "request_month"])["total_amount"]
    .sum()
    .reset_index()
)
# Optional: pivot the data to see it as a heatmap...doesnt do anything in this context of code.
    #turns the grouped table into a matrix (rows=cohort, columns=months, values=revenue)
revenue_pivot = revenue_by_cohort.pivot(index="cohort", columns="request_month", values="total_amount")

# Plot the heatmap
    #blue=revenue...lets you see what cohorts bring the most $ over time. 
plt.figure(figsize=(12, 6))
sns.heatmap(revenue_pivot, cmap="Blues", annot=True, fmt=".0f")
plt.title("Revenue by Cohort and Request Month")
plt.ylabel("Cohort Month")
plt.xlabel("Request Month")
plt.xticks(rotation=45)
plt.show()


